In [1]:
## PROCESSING LOGIC
## First download the station data for the Gulf States (TX, LA, AL, MS, FL)
## Then clip the stations to the watershed counties 
## Use the station names to build a URL and access the "narrow" sample results from the EPA Water Quality API

In [2]:
## RESOURCES

## EPA Water Quality Site:
## https://www.waterqualitydata.us/

## URL Examples
## Download all the stations for USA states FL, AL, TX, MS, LA as a GEOJSON:
## https://www.waterqualitydata.us/data/Station/search?countrycode=US&statecode=US%3A01&statecode=US%3A12&statecode=US%3A48&statecode=US%3A28&statecode=US%3A22&mimeType=geojsonv&zip=yes&providers=NWIS&providers=STEWARDS&providers=STORET

## Download all the sample data for a given station (0800257-CC-05) as a CSV:
## https://www.waterqualitydata.us/data/Result/search?siteid=0800257-CC-05&mimeType=csv&zip=yes&dataProfile=narrowResult&providers=NWIS&providers=STEWARDS&providers=STORET

## CODE REFERENCE
## Python Code reference: https://docs.python.org/3.8/
## Arcpy Reference: https://developers.arcgis.com/python/api-reference/
## Time Sleep reference: https://stackoverflow.com/questions/72590954/how-to-add-the-time-sleep-after-every-100-iterations
## Time Sleep reference 2: https://stackoverflow.com/questions/72590954/how-to-add-the-time-sleep-after-every-100-iterations
## Enumerate in Python: https://realpython.com/python-enumerate/

In [3]:
#IMPORT LIBRARIES

import arcpy
import arcgis
from arcgis.gis import GIS
#from arcgis.mapping import WebMap
from datetime import datetime
from datetime import timedelta
import urllib.request
import os
import pandas
import glob
import zipfile
gis = GIS("home")
arcpy.env.overwriteOutput = True
arcgis.env.verbose = True 

In [4]:
## DEFINE VARIABLES

ROOT = r'C:\Temp\EPA-Water-Quality' #TODO Change to your preferred directory
TODAY = (datetime.today())
CURRENT_DATE = (TODAY.strftime('%Y%m%d'))
## Testing example for Baldwin County, Alabama:
# WATER_QUALITY_STATIONS_URL = 'https://www.waterqualitydata.us/data/Station/search?countrycode=US&statecode=US%3A01&countycode=US%3A01%3A003&mimeType=geojson&zip=yes&providers=NWIS&providers=STEWARDS&providers=STORET'
## All data for AL, FL, LA, MS, TX:
# WATER_QUALITY_STATIONS_URL = 'https://www.waterqualitydata.us/data/Station/search?countrycode=US&statecode=US%3A01&statecode=US%3A12&statecode=US%3A22&statecode=US%3A28&statecode=US%3A48&mimeType=geojson&zip=yes&providers=NWIS&providers=STEWARDS&providers=STORET'
## All data within bounding box of 32N, 23N, -78W, -100W
WATER_QUALITY_STATIONS_URL ='https://www.waterqualitydata.us/data/Station/search?bBox=-100%2C23%2C-78%2C32&mimeType=csv&zip=yes&providers=NWIS&providers=STEWARDS&providers=STORET'
WORKING_DIRECTORY = ROOT + '\\working_' + str(CURRENT_DATE) + '\\'
WATER_QUALITY_STATIONS_DIRECTORY = WORKING_DIRECTORY + 'stations\\'
WATER_QUALITY_STATIONS_ZIP = WATER_QUALITY_STATIONS_DIRECTORY + 'water_quality_stations.zip'
WORKING_STATIONS_JSON = WORKING_DIRECTORY + 'station.geojson'
#WORKING_STATIONS_SHP  = WORKING_DIRECTORY + 'station.shp'
#CLIPPED_STATIONS = WORKING_DIRECTORY + 'goma_stations.shp'
COASTAL_COUNTIES = WORKING_DIRECTORY + 'coastal_watershed_counties.shp'
CSV_WORKING_DIRECTORY = WORKING_DIRECTORY + 'csv\\'
GDB_NAME = 'GoMA_Water_Quality_Data.gdb'
GDB_FULL_PATH = WORKING_DIRECTORY + GDB_NAME
WORKING_STATIONS_FC  = GDB_FULL_PATH + '\\' + 'stations'
CLIPPED_STATIONS_FC = GDB_FULL_PATH + '\\' + 'GoMA_stations'

In [5]:
## Create the directory structure

if not os.path.exists(ROOT):
    os.makedirs(ROOT)
if not os.path.exists(WORKING_DIRECTORY):
    os.makedirs(WORKING_DIRECTORY)
if not os.path.exists(CSV_WORKING_DIRECTORY):
    os.makedirs(CSV_WORKING_DIRECTORY)
if not os.path.exists(WATER_QUALITY_STATIONS_DIRECTORY):
    os.makedirs(WATER_QUALITY_STATIONS_DIRECTORY)

In [6]:
## Create a file geodatbase to output the stations and sample data to

arcpy.management.CreateFileGDB(WORKING_DIRECTORY, GDB_NAME)

<Result 'C:\\temp\\EPA-Water-Quality\\working_20230502\\GoMA_Water_Quality_Data.gdb'>

In [7]:
# Download a zipped geojson of the water quality stations for AL, FL, LA, MS and TX
# this uses the "WATER_QUALITY_STATIONS_URL" defined above

urllib.request.urlretrieve(WATER_QUALITY_STATIONS_URL, WATER_QUALITY_STATIONS_ZIP)

('C:\\Temp\\EPA-Water-Quality\\working_20230502\\stations\\water_quality_stations.zip',
 <http.client.HTTPMessage at 0x28a24a602b0>)

In [8]:
# Extract the zipped geojson stations file

with zipfile.ZipFile(WATER_QUALITY_STATIONS_ZIP, 'r') as zip_ref:
    zip_ref.extractall(WORKING_DIRECTORY)

In [9]:
# Convert the geojson stations to a feature class

arcpy.conversion.JSONToFeatures(WORKING_STATIONS_JSON, WORKING_STATIONS_FC, geometry_type="POINT")

<Result 'C:\\temp\\EPA-Water-Quality\\working_20230502\\GoMA_Water_Quality_Data.gdb\\stations'>

In [10]:
# GoMA Water Quality Area of Interest (AOI)
# ItemID = '39b57581c9de439484aa52f4fb8f44d2'

GOMA_WQ_AOI ='https://services.arcgis.com/bDAhvQYMG4WL8O5o/arcgis/rest/services/GoMA_Water_Quality_AOI/FeatureServer/4'

In [11]:
## Clip the stations to the GoMA Water Quality AOI polygon boundary 

arcpy.analysis.Clip(WORKING_STATIONS_FC, GOMA_WQ_AOI, CLIPPED_STATIONS_FC)

<Result 'C:\\temp\\EPA-Water-Quality\\working_20230502\\GoMA_Water_Quality_Data.gdb\\GoMA_stations'>

In [12]:
## Consider creating a dictionary from the Stations CSV file.
## Also, write to a Feature Class as to not loose the column names, etc.

In [13]:
arcpy.management.CalculateGeometryAttributes(CLIPPED_STATIONS_FC, [["X", "POINT_X"], ["Y", "POINT_Y"]])

<Result 'C:\\Temp\\EPA-Water-Quality\\working_20230502\\GoMA_Water_Quality_Data.gdb\\GoMA_stations'>

In [14]:
stations_df = pandas.DataFrame.spatial.from_featureclass(CLIPPED_STATIONS_FC)
stations_df.set_index('MonitoringLocationIdentifier')
stations_df.head()

,OBJECTID,ProviderName,OrganizationIdentifier,OrganizationFormalName,MonitoringLocationIdentifier,MonitoringLocationName,MonitoringLocationTypeName,ResolvedMonitoringLocationTypeName,HUCEightDigitCode,siteUrl,activityCount,resultCount,StateName,CountyName,X,Y,SHAPE
0,1,NWIS,USGS-AL,USGS Alabama Water Science Center,USGS-02376240,"DYAS CREEK AT US HWY 31 NEAR DYAS, AL.",Stream,Stream,03140106,https://www.waterqualitydata.us/provider/NWIS/...,2,51,Alabama,Baldwin County,-87.684989,30.933517,"{""x"": -87.68498889999995, ""y"": 30.933517000000..."
1,2,NWIS,USGS-AL,USGS Alabama Water Science Center,USGS-02376245,MCCURTIN CREEK NEAR BAY MINETTE AL,Stream,Stream,03140106,https://www.waterqualitydata.us/provider/NWIS/...,1,12,Alabama,Baldwin County,-87.723046,30.909073,"{""x"": -87.72304579999997, ""y"": 30.909073200000..."
2,3,NWIS,USGS-AL,USGS Alabama Water Science Center,USGS-02376250,"DYAS CREEK AT CO RD 61 NEAR BAY MINETTE, AL.",Stream,Stream,03140106,https://www.waterqualitydata.us/provider/NWIS/...,0,0,Alabama,Baldwin County,-87.639987,30.870186,"{""x"": -87.63998729999997, ""y"": 30.870185600000..."
3,4,NWIS,USGS-AL,USGS Alabama Water Science Center,USGS-02377200,STYX RIVER NEAR STAPLETON AL,Stream,Stream,03140106,https://www.waterqualitydata.us/provider/NWIS/...,1,12,Alabama,Baldwin County,-87.729157,30.738245,"{""x"": -87.72915669999998, ""y"": 30.738244800000..."
4,5,NWIS,USGS-AL,USGS Alabama Water Science Center,USGS-02377300,"STYX RIVER AT BRADY ROAD BELOW STAPLETON, AL.",Stream,Stream,03140106,https://www.waterqualitydata.us/provider/NWIS/...,23,323,Alabama,Baldwin County,-87.706656,30.691302,"{""x"": -87.70665569999994, ""y"": 30.691301500000..."


In [15]:
for col in stations_df.columns:
    print(col)

OBJECTID
ProviderName
OrganizationIdentifier
OrganizationFormalName
MonitoringLocationIdentifier
MonitoringLocationName
MonitoringLocationTypeName
ResolvedMonitoringLocationTypeName
HUCEightDigitCode
siteUrl
activityCount
resultCount
StateName
CountyName
X
Y
SHAPE


In [16]:
# create an empty table inside the file geodatabase

arcpy.management.CreateTable(
    out_path=GDB_FULL_PATH,
    out_name="GoMA_Sample_Data",
    template=r"C:\temp\EPA-Water-Quality\csvSchema.csv",
    config_keyword="",
    out_alias=""
)

<Result 'C:\\temp\\EPA-Water-Quality\\working_20230502\\GoMA_Water_Quality_Data.gdb\\GoMA_Sample_Data'>

In [17]:
# create definitions for the download routine the includes retrying if a download fails

def download_routine(URL, Output):
    retries = 3
    wait = 5
    while retries:
        try:
            urllib.request.urlretrieve(URL, Output)
            return True
        except Exception as e:
            print('failed to download, error: {}'.format(e))
            retries -=1
            if retries:
                wait *=2

In [18]:
# Use the station list to get the sample data for each station

valueList = []  # array to hold list of values collected
valueSet = set()  # set to hold values to test against to get list
rows = arcpy.SearchCursor(WORKING_STATIONS_FC)
field = "MonitoringLocationIdentifier"
index = 0
downloadFailures = []

for row in rows:
    value = row.getValue(field)
    #print(index, row)
    index += 1
    if index % 1000 == 0:
        time.sleep(60)
        print("60-second pause.....")
        print('Processing Station ' + value)
    old_file = os.path.join(CSV_WORKING_DIRECTORY, 'narrowresult.csv')
    new_file = os.path.join(CSV_WORKING_DIRECTORY, value +'.csv')
    sample_data_zip = CSV_WORKING_DIRECTORY + str(value) + '_' + CURRENT_DATE + '_sample_narrow.zip'
    station_narrow_url = 'https://www.waterqualitydata.us/data/Result/search?siteid=' + value + '&mimeType=csv&zip=yes&dataProfile=narrowResult&providers=NWIS&providers=STEWARDS&providers=STORET'
    if download_routine(station_narrow_url, sample_data_zip):
        with zipfile.ZipFile(sample_data_zip, 'r') as zip_ref:
            zip_ref.extractall(CSV_WORKING_DIRECTORY)
        os.rename(old_file, new_file)
        os.remove(sample_data_zip)
        sample_df = pandas.read_csv(new_file)
        sample_df.set_index('MonitoringLocationIdentifier')
        result = pandas.merge(sample_df, stations_df, how='left')
        result.drop(columns=['OBJECTID','SHAPE'], inplace=True)
        result.to_csv(new_file)
    else:
        downloadFailures.append(value)
    ## Load stations list as a dictionary would be an improvement here
    fcName = value.replace('-','_')
    # OFC = GDB_FULL_PATH + '\\' + fcName,
    # print(OFC)
    # arcpy.management.Append(
    #     inputs=new_file,
    #     target=GDB_FULL_PATH +'\\' + 'GoMA_Sample_Data',
    #     schema_type="TEST",
    #     field_mapping=None,
    #     subtype="",
    #     expression="",
    #     match_fields=None,
    #     update_geometry="NOT_UPDATE_GEOMETRY"
    # )
    #arcpy.TableToTable_conversion(new_file, GDB_FULL_PATH, fcName)
    # arcpy.management.XYTableToPoint(
    #     in_table=new_file,
    #     out_feature_class= OFC,
    #     x_field="X",
    #     y_field="Y",
    #     z_field=None,
    #     coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
    #     )
print('Number of download failures: {}'.format(len(downloadFailures)))
print(downloadFailures)



C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\1235162711.py:27: DtypeWarning: Columns (10,60,62,63,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pandas.read_csv(new_file)
C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\1235162711.py:27: DtypeWarning: Columns (10,62,63,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pandas.read_csv(new_file)
C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\1235162711.py:27: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pandas.read_csv(new_file)
C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\1235162711.py:27: DtypeWarning: Columns (69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pandas.read_csv(new_file)
C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\1235162711.py:27: DtypeWarning: Columns (9,20,71) have mixe

60-second pause.....
Processing Station 21AWIC-886


C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\1235162711.py:27: DtypeWarning: Columns (17,60) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pandas.read_csv(new_file)


Number of download failures: 0
[]


In [19]:
# now let's merge all the CSV files into one

SAMPLE_NARROW_MERGED_CSV = CSV_WORKING_DIRECTORY + "All_GOMA_Sample_Narrow_Results.csv"
joined_files = os.path.join(CSV_WORKING_DIRECTORY, "*.csv")
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
# Finally, the files are joined
GOMA_NAROW_STATIONS_MERGED = pandas.concat(map(pandas.read_csv, joined_list), ignore_index=True)
GOMA_NAROW_STATIONS_MERGED.to_csv(SAMPLE_NARROW_MERGED_CSV, index=False)

C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\3856583867.py:8: DtypeWarning: Columns (11,61,63,64,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  GOMA_NAROW_STATIONS_MERGED = pandas.concat(map(pandas.read_csv, joined_list), ignore_index=True)
C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\3856583867.py:8: DtypeWarning: Columns (11,63,64,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  GOMA_NAROW_STATIONS_MERGED = pandas.concat(map(pandas.read_csv, joined_list), ignore_index=True)
C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\3856583867.py:8: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  GOMA_NAROW_STATIONS_MERGED = pandas.concat(map(pandas.read_csv, joined_list), ignore_index=True)
C:\Users\keit8223\AppData\Local\Temp\1\ipykernel_40396\3856583867.py:8: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set

In [20]:
# now let's create a shapefile from the merged CSV file

# SHAPEFILE_NAME = 'sample_stations'
FC_NAME = 'sample_stations'
# SAMPLE_STATIONS_SHP = CSV_WORKING_DIRECTORY + SHAPEFILE_NAME + '.shp'
SAMPLE_STATIONS_FC = GDB_FULL_PATH + '//' + FC_NAME
# SAMPLE_STATIONS = CSV_WORKING_DIRECTORY + SHAPEFILE_NAME

arcpy.management.XYTableToPoint(
    in_table=SAMPLE_NARROW_MERGED_CSV,
    out_feature_class=SAMPLE_STATIONS_FC,
    x_field="X",
    y_field="Y",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\temp\\EPA-Water-Quality\\working_20230502\\GoMA_Water_Quality_Data.gdb\\sample_stations'>

In [21]:
# # now let's zip the shapefile so we can publish to arcgis online

# ZIPFILE_LOCATION = CSV_WORKING_DIRECTORY + SHAPEFILE_NAME + '.zip'

# shapefiles_list = [SAMPLE_STATIONS + ".cpg",
#                    SAMPLE_STATIONS + ".dbf",
#                    SAMPLE_STATIONS + ".prj",
#                    SAMPLE_STATIONS + ".sbn",
#                    SAMPLE_STATIONS + ".sbx",
#                    SAMPLE_STATIONS + ".shp",
#                    SAMPLE_STATIONS + ".shp.xml",
#                    SAMPLE_STATIONS + ".shx"]
# with zipfile.ZipFile(ZIPFILE_LOCATION, 'w') as zipMe:        
#     for file in shapefiles_list:
#         zipMe.write(file, compress_type=zipfile.ZIP_DEFLATED)